# AI Data Engineering Agent (Master Edition)

This is a **True Full-Scan Agent** that meets the 10-Point Master Checklist.
It does NOT just summarize data. It reads **every single row** (in chunks), detects hidden issues (like 'Delhi#2024'), writes production-grade code, executes it, and hands you a finished notebook.

## 1. Setup & Dependencies
Ensure you have the required packages installed:
`!pip install langchain langchain-google-genai pandas numpy sqlalchemy joblib scikit-learn nbformat`

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import joblib
import nbformat as nbf
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate

# --- CONFIGURATION ---
# Please set your GOOGLE API key here (Get it from aistudio.google.com)
os.environ["GOOGLE_API_KEY"] = "AIza..." 

# Using Gemini Pro for its large context window (Critical for accumulating scan insights)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

## 2. Module 1: The Chunk Scanner (Deep Awareness)
This function iterates through the dataset in chunks, converting rows to JSON. 
It asks Gemini to identify specific defects in each chunk that summary stats would miss.

In [ ]:
def scan_dataset(df, chunk_size=300):
    """
    Scans the ENTIRE dataset in chunks of `chunk_size`.
    Returns a aggregated list of issues found by Gemini.
    """
    print(f"Starting Full-Scan on {len(df)} rows...")
    
    all_issues = []
    
    chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]
    
    prompt_template = ChatPromptTemplate.from_template("""
    Analyze this snippet of raw data rows (JSON format):
    {data_chunk}
    
    Identify SPECIFIC data quality issues. Look for:
    1. Mixed types in the same column (e.g. strings in numeric cols).
    2. Hidden delimiters (e.g. 'Delhi#2024' or 'John|Doe').
    3. Dirty formats (Date variations like '12-01-2020' vs 'Jan 12, 2020').
    4. Outliers or impossible values (Age = 150).
    
    Return ONLY a concise bulleted list of issues found. If none, say 'No major issues'.
    """)
    
    for i, chunk in enumerate(chunks):
        # Convert chunk to JSON for LLM readability
        chunk_json = chunk.to_json(orient='records')
        
        # FULL SCAN: No Break. We iterate everything.
        # Checklist Item #1: "The agent must read the ENTIRE dataset"
            
        print(f"Scanning Chunk {i+1}/{len(chunks)}...")
        chain = prompt_template | llm
        result = chain.invoke({"data_chunk": chunk_json})
        
        if "No major issues" not in result.content:
            all_issues.append(f"Chunk {i+1} Issues:\n{result.content}")
            
    print("Scan Complete.")
    return "\n".join(all_issues)

## 3. Module 2: The Code Architect (Pipeline Generation)
Takes the scan report and basic dataframe info to write the Cleaning Code.

In [ ]:
def generate_cleaning_code(df, scan_report):
    """
    Generates executable Python code to fix the issues identified in the scan.
    """
    print("Architecting Data Pipeline...")
    
    dtypes_info = df.dtypes.to_string()
    head_info = df.head().to_string()
    
    prompt = f"""
    You are a Senior Data Engineer. 
    
    I have scanned the dataset and found these issues:
    {scan_report}
    
    Dataset Head:
    {head_info}
    
    Types:
    {dtypes_info}
    
    TASKS:
    1. Create a python script to CLEAN this dataset.
    2. Handle missing values (Impute numericals with mean/median, categoricals with mode).
    3. Fix identified issues (e.g. split columns if needed, convert types).
    4. Encode Categoricals (LabelEncoder) and Scale Numericals (StandardScaler).
    5. SAVE artifacts: 'clean_data.csv', 'model.pkl' (dummy), 'preprocessor.pkl'.
    
    CRITICAL RULES:
    - The code must assume `df` is already loaded.
    - Do NOT re-load the data.
    - Use `joblib` to save the scaler/encoders.
    - Return ONLY the python code block wrapped in ```python ... ```.
    """
    
    result = llm.invoke(prompt)
    return result.content

## 4. Module 3: In-Memory Execution
Runs the generated code using `exec()` to modify the dataframe in real-time.

In [ ]:
def execute_pipeline(df, code):
    """
    Executes the cleaning code on the DataFrame.
    """
    import re
    
    # Extract code info
    code_match = re.search(r"```python(.*?)```", code, re.DOTALL)
    if code_match:
        clean_code = code_match.group(1)
    else:
        clean_code = code
        
    print("--- Executing Generated Pipeline ---")
    print(clean_code)
    print("------------------------------------")
    
    # Prepare execution environment
    # We pass 'df' into the exec scope so the code can modify it.
    exec_globals = globals().copy()
    exec_globals['df'] = df
    exec_globals['pd'] = pd
    exec_globals['np'] = np
    exec_globals['joblib'] = joblib
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    exec_globals['StandardScaler'] = StandardScaler
    exec_globals['LabelEncoder'] = LabelEncoder
    
    try:
        exec(clean_code, exec_globals)
        print("\nExecution Success! Artifacts (clean_data.csv, etc.) should be saved.")
        # Retrieve modified df if needed, though clean_data.csv is the source of truth now
        return True
    except Exception as e:
        print(f"EXECUTION FAILED: {e}")
        return False

## 5. Module 4: The Handover (User Notebook Gen)
Creates a ready-to-use notebook for the user, loading the *cleaned* data.

In [ ]:
def create_user_notebook():
    nb = nbf.v4.new_notebook()
    
    nb.cells.append(nbf.v4.new_markdown_cell("# AI Engineered Data Notebook\nThis notebook loads your professionally cleaned dataset."))
    
    code = """
import pandas as pd
import joblib

# 1. Load Cleaned Data
df = pd.read_csv('clean_data.csv')
print('Loaded Cleaned Data:', df.shape)

# 2. Load Preprocessors
try:
    preprocessor = joblib.load('preprocessor.pkl')
    print('Loaded Preprocessor:', preprocessor)
except:
    print('No preprocessor found (maybe none were needed).')

display(df.head())
"""
    nb.cells.append(nbf.v4.new_code_cell(code))
    
    nb.cells.append(nbf.v4.new_markdown_cell("## Next Steps\nYou can now run your ML models below."))
    nb.cells.append(nbf.v4.new_code_cell("# Your Model Here\n# from sklearn.ensemble import RandomForestClassifier..."))
    
    with open('AI_Data_Engineered_Notebook.ipynb', 'w') as f:
        nbf.write(nb, f)
    print("Created 'AI_Data_Engineered_Notebook.ipynb' for the user.")

## 6. The Master Agent Runner
Orchestrates the entire flow.

In [ ]:
def run_data_engineering_agent(df):
    if df is None:
        return "No data."
    
    # 1. Scan
    scan_report = scan_dataset(df)
    
    # 2. Architect
    code = generate_cleaning_code(df, scan_report)
    
    # 3. Execute
    success = execute_pipeline(df, code)
    
    # 4. Handover
    if success:
        create_user_notebook()
        print("Done. Open 'AI_Data_Engineered_Notebook.ipynb' to continue.")
    else:
        print("Failed to create user notebook due to execution errors.")

## 7. Test Drive
Let's create a nasty dataset with hidden issues.

In [ ]:
# Dirty Data
data = {
    'ID': [1, 2, 3, 4, 5],
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'Age': [25, 30, 150, 40, 22], # 150 is outlier
    'City_Year': ['NY#2021', 'LA#2020', 'SF#2022', 'NY#2021', 'Austin#2023'], # Hidden Delimiter
    'Salary': ['50k', '60000', '75k', '90000', 'None'] # Mixed types and strings
}
df_dirty = pd.DataFrame(data)

print("Original Dirty Data:")
print(df_dirty)

# Go!
run_data_engineering_agent(df_dirty)